# Import Libraries, Read in .json

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import date

from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer


df = pd.read_json('C:/Users/secre/Downloads/evaluate_news.json')
print(df.shape)
df.head()

KeyboardInterrupt: 

# Preprocessing, Developing New Columns
Get the labels, find the change from start price day 1 to end of day 3. 
Drop unavailable ticker data.

In [ ]:

#Get the labels, normalize them, and concatenate to the dataframe. Then, add a new column for the overall baseline change from start day 1 to end of day 3.
labels = df["labels"]
labels = pd.json_normalize(labels)
data = pd.concat([df,labels],axis=1)
data["baselinePctChng3"] = ((data["end_price_3day"] - data["start_price_open"]) / data["start_price_open"])*100
data.dropna()

,text,title,pub_time,labels,ticker,start_time,start_price_open,start_price_close,end_price_1day,end_price_2day,...,highest_time_1day,highest_time_2day,highest_time_3day,lowest_price_1day,lowest_price_2day,lowest_price_3day,lowest_time_1day,lowest_time_2day,lowest_time_3day,baselinePctChng3
0,"PLANO, Texas, Dec. 8, 2020 /PRNewswire/ --Euro...",European Wax Center Welcomes Jennifer Vanderve...,2020-12-08 09:00:00-05:00,"{'ticker': 'MIK', 'start_time': '2020-12-08 09...",MIK,2020-12-08 09:00:00-05:00,12.070,12.070,12.80,12.4899,...,2020-12-08 10:12:00-05:00,2020-12-08 10:12:00-05:00,2020-12-08 10:12:00-05:00,11.98,11.9800,11.9800,2020-12-08 09:13:00-05:00,2020-12-08 09:13:00-05:00,2020-12-08 09:13:00-05:00,7.705054
1,"CHARLOTTE, N.C., Oct. 1, 2020 /PRNewswire/ --D...","Duke Energy to host virtual Environmental, Soc...",2020-10-01 12:11:00-04:00,"{'ticker': 'DUK', 'start_time': '2020-10-01 12...",DUK,2020-10-01 12:11:00-04:00,89.740,89.780,90.05,91.0000,...,2020-10-01 15:21:00-04:00,2020-10-02 15:32:00-04:00,2020-10-02 15:32:00-04:00,89.18,88.6500,88.6500,2020-10-01 14:03:00-04:00,2020-10-02 09:29:00-04:00,2020-10-02 09:29:00-04:00,2.518386
2,"SOUTH SAN FRANCISCO, Calif., Oct. 5, 2020 /PRN...",Pliant Therapeutics Appoints Mike Ouimette as ...,2020-10-05 08:00:00-04:00,"{'ticker': 'PLRX', 'start_time': '2020-10-05 0...",PLRX,2020-10-05 09:29:00-04:00,20.000,20.000,21.43,21.9200,...,2020-10-05 15:50:00-04:00,2020-10-06 09:35:00-04:00,2020-10-07 15:28:00-04:00,19.73,19.7300,19.7300,2020-10-05 09:57:00-04:00,2020-10-05 09:57:00-04:00,2020-10-05 09:57:00-04:00,21.750000
4,"ENGLEWOOD, Colo., Feb. 22, 2021 /PRNewswire/ -...","DISH Network reports fourth quarter, year-end ...",2021-02-22 06:05:00-05:00,"{'ticker': 'DISH', 'start_time': '2021-02-22 0...",DISH,2021-02-22 06:09:00-05:00,34.250,34.250,32.00,30.8500,...,2021-02-22 06:37:00-05:00,2021-02-22 06:37:00-05:00,2021-02-22 06:37:00-05:00,31.25,29.7100,29.7100,2021-02-22 18:54:00-05:00,2021-02-23 12:09:00-05:00,2021-02-23 12:09:00-05:00,-8.321168
5,"STAMFORD, Conn., Jan. 21, 2021 /PRNewswire/ --...",Targetspot to Integrate Tru Optik Data Managem...,2021-01-21 07:30:00-05:00,"{'ticker': 'TGT', 'start_time': '2021-01-21 08...",TGT,2021-01-21 08:00:00-05:00,188.350,189.000,191.00,191.7501,...,2021-01-21 10:05:00-05:00,2021-01-21 10:05:00-05:00,2021-01-25 08:00:00-05:00,188.35,188.3500,188.3500,2021-01-21 08:00:00-05:00,2021-01-21 08:00:00-05:00,2021-01-21 08:00:00-05:00,0.902575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303882,"--(BUSINESS WIRE)--()--PPD, Inc.PPD Great Plac...",PPD 2020TM,2020-12-07 20:00:00-05:00,"{'ticker': 'PPD', 'start_time': '2020-12-08 09...",PPD,2020-12-08 09:30:00-05:00,35.570,35.730,34.94,34.6500,...,2020-12-08 09:30:00-05:00,2020-12-08 09:30:00-05:00,2020-12-08 09:30:00-05:00,34.55,34.4644,34.4644,2020-12-08 11:46:00-05:00,2020-12-09 13:58:00-05:00,2020-12-09 13:58:00-05:00,-1.208884
303883,LONDON--(BUSINESS WIRE)-- FORM 8.3 PUBLIC OPEN...,Form 8.3 - TTWO US,2020-11-24 10:00:00-05:00,"{'ticker': 'TTWO', 'start_time': '2020-11-24 1...",TTWO,2020-11-24 10:00:00-05:00,169.565,169.500,169.90,174.2000,...,2020-11-24 14:35:00-05:00,2020-11-25 19:09:00-05:00,2020-11-27 10:32:00-05:00,167.29,167.2900,167.2900,2020-11-24 10:53:00-05:00,2020-11-24 10:53:00-05:00,2020-11-24 10:53:00-05:00,4.036800
303886,"DUBLIN--(BUSINESS WIRE)--The ""Motorcycle Slip-...",Global Motorcycle Slip-on Exhaust System Marke...,2020-12-22 12:56:00-05:00,"{'ticker': 'DRIV', 'start_time': '2020-12-22 1...",DRIV,2020-12-22 12:56:00-05:00,23.230,23.230,23.50,23.4000,...,2020-12-22 17:10:00-05:00,2020-12-23 08:01:00-05:00,2020-12-24 09:01:00-05:00,23.23,23.2300,23.2300,2020-12-22 12:56:00-05:00,2020-12-22 12:56:00-05:00,2020-12-22 12:56:00-05:00,1.592768
303889,NEW YORK--(BUSINESS WIRE)--Metropolitan Commer...,Metropolitan Commercial Bank Appoints Ram Pupp...,2021-01-13 06:00:00-05:00,"{'ticker': 'STT', 'start_time': '2021-01-13 08...",STT,2021-01-13 08:0

In [ ]:
data.head()

,text,title,pub_time,labels,ticker,start_time,start_price_open,start_price_close,end_price_1day,end_price_2day,...,highest_time_1day,highest_time_2day,highest_time_3day,lowest_price_1day,lowest_price_2day,lowest_price_3day,lowest_time_1day,lowest_time_2day,lowest_time_3day,baselinePctChng3
0,"PLANO, Texas, Dec. 8, 2020 /PRNewswire/ --Euro...",European Wax Center Welcomes Jennifer Vanderve...,2020-12-08 09:00:00-05:00,"{'ticker': 'MIK', 'start_time': '2020-12-08 09...",MIK,2020-12-08 09:00:00-05:00,12.07,12.07,12.80,12.4899,...,2020-12-08 10:12:00-05:00,2020-12-08 10:12:00-05:00,2020-12-08 10:12:00-05:00,11.98,11.98,11.98,2020-12-08 09:13:00-05:00,2020-12-08 09:13:00-05:00,2020-12-08 09:13:00-05:00,7.705054
1,"CHARLOTTE, N.C., Oct. 1, 2020 /PRNewswire/ --D...","Duke Energy to host virtual Environmental, Soc...",2020-10-01 12:11:00-04:00,"{'ticker': 'DUK', 'start_time': '2020-10-01 12...",DUK,2020-10-01 12:11:00-04:00,89.74,89.78,90.05,91.0000,...,2020-10-01 15:21:00-04:00,2020-10-02 15:32:00-04:00,2020-10-02 15:32:00-04:00,89.18,88.65,88.65,2020-10-01 14:03:00-04:00,2020-10-02 09:29:00-04:00,2020-10-02 09:29:00-04:00,2.518386
2,"SOUTH SAN FRANCISCO, Calif., Oct. 5, 2020 /PRN...",Pliant Therapeutics Appoints Mike Ouimette as ...,2020-10-05 08:00:00-04:00,"{'ticker': 'PLRX', 'start_time': '2020-10-05 0...",PLRX,2020-10-05 09:29:00-04:00,20.00,20.00,21.43,21.9200,...,2020-10-05 15:50:00-04:00,2020-10-06 09:35:00-04:00,2020-10-07 15:28:00-04:00,19.73,19.73,19.73,2020-10-05 09:57:00-04:00,2020-10-05 09:57:00-04:00,2020-10-05 09:57:00-04:00,21.750000
3,"NEW YORK, Jan. 18, 2021 /PRNewswire/ -- With a...",Radiofrequency Ablation Devices Market Revenue...,2021-01-18 02:30:00-05:00,{'ticker': ''},,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"ENGLEWOOD, Colo., Feb. 22, 2021 /PRNewswire/ -...","DISH Network reports fourth quarter, year-end ...",2021-02-22 06:05:00-05:00,"{'ticker': 'DISH', 'start_time': '2021-02-22 0...",DISH,2021-02-22 06:09:00-05:00,34.25,34.25,32.00,30.8500,...,2021-02-22 06:37:00-05:00,2021-02-22 06:37:00-05:00,2021-02-22 06:37:00-05:00,31.25,29.71,29.71,2021-02-22 18:54:00-05:00,2021-02-23 12:09:00-05:00,2021-02-23 12:09:00-05:00,-8.321168


In [ ]:
data.shape

(303893, 27)

# Develop the Corpus
Entries: 57,031 total with positive, 48,955 total entries with negative.


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer 


In [69]:
dataShuffled = data.sample(frac=1)
dataPos = dataShuffled[dataShuffled["baselinePctChng3"] > 0]
dataNeg = dataShuffled[dataShuffled["baselinePctChng3"] < 0]
# print(len(dataPos))
# print(len(dataNeg))

posX = dataPos.sample(frac = 10000/57031, random_state=1)
negX = dataNeg.sample(frac = 10000/48955,random_state=1)
# print(posX.shape)
# print(negX.shape)

# Determine most important words using TF-IDF w/transformer, TF-IDF w/vectorizer, and purely based on frequency (no model, just math)

typically, If you need the term frequency (term count) vectors for different tasks, use transformer. If you need to compute tf-idf scores on documents within your “training” dataset, use vectorizer.

Note: results got the same 'top' words.

### Positives

In [54]:
#TF-IDF with transformer

sentencesPositive = []
wordsPositive = []

for index, row in posX.iterrows():
    sentence = row['title']
    sentencesPositive.append(sentence)

print(len(sentencesPositive))

countVector = CountVectorizer()
wordCountVector = countVector.fit_transform(sentencesPositive)

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(wordCountVector)

matrix = countVector.transform(sentencesPositive)
tf_idf_vector=tfidf_transformer.transform(matrix)

feature_names = countVector.get_feature_names_out() 
first_document_vector=tf_idf_vector[0] 

df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

10000


,tfidf
bankers,0.487884
timing,0.432129
community,0.319160
trust,0.308155
release,0.287665
...,...
endorsed,0.000000
endorse,0.000000
endometrial,0.000000
endocrinology,0.000000


In [55]:
posDict = dict(zip(df.index, df['tfidf']))

In [59]:
#TF-IDF with vectorizer
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

# settings that you use for count vectorizer will go here 
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 

#transform, send in
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(sentencesPositive)
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0] 

#place in dataframe, sort by highest weight.
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names_out(), columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
bankers,0.487884
timing,0.432129
community,0.319160
trust,0.308155
release,0.287665
...,...
endorsed,0.000000
endorse,0.000000
endometrial,0.000000
endocrinology,0.000000


### Negatives

In [70]:
#TF-IDF with transformer
sentencesNegative = []
wordsPositive = []

for index, row in negX.iterrows():
    sentence = row['title']
    sentencesNegative.append(sentence)

print(len(sentencesNegative))

countVector = CountVectorizer()
wordCountVector = countVector.fit_transform(sentencesNegative)

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(wordCountVector)

matrix = countVector.transform(sentencesNegative)
tf_idf_vector=tfidf_transformer.transform(matrix)

feature_names = countVector.get_feature_names_out() 
first_document_vector=tf_idf_vector[0] 

df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

10000


,tfidf
hallmark,0.370480
hall,0.332010
violations,0.324355
hagens,0.317883
berman,0.307332
...,...
emre,0.000000
empowers,0.000000
empowerment,0.000000
empowering,0.000000


In [46]:
negDict = dict(zip(df.index, df['tfidf']))

In [71]:
#TF-IDF with vectorizer

# settings that you use for count vectorizer will go here 
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 

# just send in all your docs here 
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(sentencesNegative)
# get the first vector out (for the first document) 
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0] 

# place tf-idf values in a pandas data frame 
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names_out(), columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
hallmark,0.370480
hall,0.332010
violations,0.324355
hagens,0.317883
berman,0.307332
...,...
emre,0.000000
empowers,0.000000
empowerment,0.000000
empowering,0.000000


We now have 2 dictionaries with word - TF-IDF pairings. TF-IDF scores range from 0 --> 1, where 1 is a heavy importance weighting and 0 is less important.

Now that we have the top words for both positive and negative, we check for overlap with a threshold of .1 in the TF-IDF to see which words are important to a stock's drop and rise but also who's meanings depend on the surrounding words. Such words would be thrown into a model such as 'BERT' for future reference.

In [47]:
wordsInBoth = []
for key in posDict.keys():
    if key in negDict.keys() and posDict[key] > 0.2 and negDict[key] > 0.1:
        wordsInBoth.append(key)

In [48]:
print(wordsInBoth)

[]


Only Therapeutics appears on both with a threshold of 0.1, which alludes to there being a 'clear' division between use of words and their related stock price changes.

One major consideration is that each word comes from a different industry; one could extrapolate the causation when referring back to the original media and seeing what type of the 11 reports it was.

### Preliminary work

- first cell identifies the most important word in each sentence.
- second cell identifies a 'non' TF-IDF method of determining word frequency and most important words. It was determined not as powerful as the TF-IDF models.

In [ ]:
#OTHER:
# # Identify the most important word in each sentence
# for sentence in cleanSentences:
#     words = sentence.split()
#     word_scores = {}
#     for word in words:
#         # Calculate word score as the product of its frequency and length
#         score = len(word) * words.count(word)
#         word_scores[word] = score
#     mostImportantWordInSentence = max(word_scores, key=word_scores.get)
#     importantWords.append(mostImportantWordInSentence)
#     print(mostImportantWordInSentence)

In [ ]:
#most important words without TF-IDF
# nltk.download('stopwords')
# stopWords = set(stopwords.words('english'))

# cleanSentences = []
# importantWords = []
# for sentence in sentencesPositive: #SWAP FOR SENTENCESNEGATIVE.
#     words = sentence.split()
#     filteredWords = [word for word in words if word.lower() not in stopWords]
#     cleanSentence = ' '.join(filteredWords)
#     cleanSentences.append(cleanSentence)


# wordCounter = {}
# for sentence in cleanSentences:
#     words = sentence.split()
#     word_scores = {}
#     for word in words:
#         # Calculate word score as the product of its frequency and length
#         score = len(word) * words.count(word)
#         word_scores[word] = score
#     most_important_word = max(word_scores, key=word_scores.get)
#     if most_important_word in wordCounter:
#         wordCounter[most_important_word] += 1
#     else:
#         wordCounter[most_important_word] = 1

# for sentence, count in wordCounter.items():
#     print(f"{sentence}: {count}")


# # Remove key-value pairings where the key contains the substring ".com"
# to_delete = []
# for key in wordCounter.keys():
#     if ".com" in key or "https:" in key:
#         to_delete.append(key)
# for key in to_delete:
#     del wordCounter[key]


# top_words = sorted(wordCounter.items(), key=lambda x: x[1], reverse=True)[:20]

# print(top_words)